# 실전크롤링 - 4. 카카오 쇼핑 톡딜 크롤링
- 상품평, 상세페이지 링크, 판매가격, 판매처, 배송정보를 엑셀로 정리
- 5번만 스크롤하여 총 300개 상품내역 가져오기

# 1) 정적 vs. 동적 구분
- 동적 페이지임

In [41]:
from selenium import webdriver

driver = webdriver.Chrome()
driver.get('https://shoppinghow.kakao.com/siso/p/sale/mall/talkstore')

# 2) 5번 스크롤 내리기

In [42]:
import time
for i in range(5):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(1)

# 3) 크롤링

In [43]:
from bs4 import BeautifulSoup

#html 추출
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser') 

# 크롤링
tags = soup.select('.list_spsale.__tiara_container_ > li')  #태그 안에 공백 존재

output = []
for tag in tags:
    name = tag.select_one('.info_name').text
    link = 'https://shoppinghow.kakao.com' + tag.select_one('.link_product').attrs['href'] #링크찾기? #프로토콜, 도메인추가
    price = tag.select_one('.info_sales').text.replace(',','').replace('원','') #숫자만 남기기
    price = int(price)
    store = tag.select_one('.txt_shop').text
    dilivery = tag.select_one('.info_etc').contents[-1]  #(주의) 상위 클래스에 속해있음
    output.append([name, link, price, store, dilivery])

import pandas as pd
df = pd.DataFrame(data=output, columns=['name', 'link','price', 'store', 'dilivery'])
df.to_excel('kakao_shopping.xlsx', index=False)